## Draft

## Bayesian Optimization

In [46]:
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
from bayes_opt import BayesianOptimization
output_notebook()

Loading BokehJS ...

In [171]:
bayopt = BayesianOptimizer(kappa=3)
bayopt.initialize()
bayopt.make_plot()
bayopt.optimize(n_iter=7)

In [167]:
class BayesianOptimizer:
    
    def __init__(self, kappa):
        self.kappa = kappa
    
    def target(self, x):
        '''Define target function to be optimize'''
        return np.exp(-(x - 2)**2) + np.exp(-(x - 6)**2/10) + 1/ (x**2 + 1)
        
    def initialize(self):
        '''Initialize Bayesian Optimization'''
        self.x_lower = -5
        self.x_upper = 10
        self.bo = BayesianOptimization(f=self.target, verbose=0,
                                       pbounds={'x': (self.x_lower, self.x_upper)})
        
    def optimize(self, n_iter, **kwargs):
        '''Perform bayesian optimization'''
        for i in range(n_iter):
            self.bo.maximize(init_points=2, n_iter=1, acq='ucb', kappa=self.kappa)
            self.update_plot()
            
    def prepare_data(self):
        '''Prepares data to create plot'''
        x = np.arange(self.x_lower, self.x_upper, .01)
        y = self.target(x)
        mean, s = self.bo.gp.predict(x.reshape(-1, 1), return_std=True)
        s_upper = mean + s
        s_lower = mean - s
        x = np.hstack((x,x[::-1]))
        y = np.hstack((y,y[::-1]))
        sigma = np.hstack((s_upper, s_lower[::-1]))
        mean = np.hstack((mean,mean[::-1]))
        return x, y, mean, sigma
    
    def make_plot(self):
        '''Initialize Bokeh plot'''
        x, y, mean, sigma = self.prepare_data()
        self.source = ColumnDataSource(dict(x=x, sigma=sigma, mean=mean, y=y))
        plot = figure(plot_width=700, plot_height=400)
        plot.patch(x='x', y='sigma',
                    source=self.source, fill_color='lightgreen',
                    line_color='green', line_width=0.1)
        plot.line(x='x', y='mean', source=self.source,
                 line_color = 'black', line_dash='dashed')
        plot.line(x='x', y='y', source=self.source,
                  line_color='red', line_width = 2)
        
        utility = np.zeros_like(x)
        self.source_U = ColumnDataSource(dict(x=x, utility=utility))
        plot_U = figure(plot_width=700, plot_height=250)
        plot_U.line(x='x', y='utility', source=self.source_U)
        
        P = column(plot, plot_U)
        self.handle = show(P, notebook_handle=True)
 
    
    def update_plot(self):
        '''Updates Bokeh plot'''
        x, y, mean, sigma = self.prepare_data()
        self.source.data  = {'x' : x, 'y' : y, 'mean' : mean, 'sigma' : sigma}
        utility = self.bo.util.utility(x.reshape(-1, 1), self.bo.gp, 0) 
        push_notebook(handle=self.handle)
        self.source_U.data = {'x' : x, 'utility': utility}
        push_notebook(handle=self.handle)